In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision.datasets import MNIST
from torchvision.transforms import transforms as T

from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

import numpy as np
from tqdm.notebook import tqdm

!git clone https://github.com/taeuk0210/torch-study
%cd /content/torch-study/Ch01/ex02

from utils import print_image_from_tensor, calc_accuracy, print_image_from_path, image_to_tensor

In [6]:
transform = T.Compose([
    T.Resize((20, 20)),
    T.ToTensor()
])

trainset = MNIST(root="./train", train=True, download=True, transform=transform)
testset = MNIST(root="./test", train=False, download=True, transform=transform)

train_loader = DataLoader(trainset, batch_size=32, shuffle=True)
test_loader = DataLoader(testset, batch_size=32, shuffle=False)

In [28]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(400, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),

            nn.Linear(64, 10)
        )

        self._init_weights(self.layers)

    def forward(self, x):
        return self.layers(x)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)

        elif isinstance(m, nn.BatchNorm1d):
            nn.init.ones_(m.weight)
            nn.init.zeros_(m.bias)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 5
lr = 0.025

model = MyModel()
optim = Adam(model.parameters(), lr=lr)

model.to(device)
for epoch in tqdm(range(1, num_epochs + 1)):

    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []

    for (x, y) in train_loader:

        x = x.flatten(1).to(device)
        y = y.to(device)

        optim.zero_grad()

        y_pred = model(x)

        loss = F.cross_entropy(y_pred, y)

        loss.backward()

        optim.step()

        train_loss.append(loss.detach().cpu().item())
        train_acc.append(calc_accuracy(y_pred, y))

    model.eval()
    with torch.no_grad():

        for (x, y) in test_loader:

            x = x.flatten(1).to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = F.cross_entropy(y_pred, y)

            test_loss.append(loss.detach().cpu().item())
            test_acc.append(calc_accuracy(y_pred, y))


    print(f"[Epoch : {epoch:2d}]" +
          f"[Train Loss : {np.mean(train_loss):2.6f}]" +
          f"[Test Loss : {np.mean(test_loss):2.6f}]" +
          f"[Train Acc : {np.mean(train_acc):1.4f}]" +
          f"[Test Acc : {np.mean(test_acc):1.4f}]")


In [ ]:
test_image = image_to_tensor("./images/test5.jpg")
print_image_from_tensor(test_image)

model.eval()
pred = model(test_image.reshape(1, -1).to(device))

pred.softmax(1).argmax()